In [1]:
import run_info_utils

In [2]:
def get_partial_df(df, n_rows):
    if df.shape[0] < n_rows:
        return df
    else:
        return df[:n_rows]
    
def preprocess(df):
    h_dim_list = list()
    for i in range(df.shape[0]):
        h_dim = df['hidden_dimensions'].iloc[i][0]
        h_dim_list.append(h_dim)
    df['hidden_dimension'] = h_dim_list
    return df

In [3]:
df = run_info_utils.get_df_run_info()

In [4]:
df = preprocess(df)

In [5]:
df.head(n=1)

,all_correct_stop,batch_size,carry_list,confidence_prob,dev/carry-0/early_stopping/accuracy,dev/carry-1/early_stopping/accuracy,dev/carry-2/early_stopping/accuracy,dev/carry-3/early_stopping/accuracy,dev/carry-4/early_stopping/accuracy,dev/early_stopping/max_correct_answer_step,...,time/running_time,time/start_time,train_dev_test_ratio,train_set_size,train_set_size/carry-0,train_set_size/carry-1,train_set_size/carry-2,train_set_size/carry-3,train_set_size/carry-4,hidden_dimension
0,True,32,"[0, 1, 2, 3, 4]",0.9,1.0,1.0,1.0,1.0,1.0,4.0,...,00:39:58.418681,2019-03-25 20:29:15.231582,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0,24


In [6]:
#print(list(df.columns))

In [7]:
experiment_name = 'iccm2019'
df = df.loc[df['experiment_name'] == experiment_name]

cols = ['run_id', 'operator', 'rnn_type', 'confidence_prob', 
        'operand_bits', 'hidden_activation', 'max_steps', 'hidden_dimension',
'dev/last_carry-0_mean_correct_answer_step', 
'dev/last_carry-1_mean_correct_answer_step', 
'dev/last_carry-2_mean_correct_answer_step', 
'dev/last_carry-3_mean_correct_answer_step', 
'dev/last_carry-4_mean_correct_answer_step', 
'dev/last_mean_correct_answer_step', 'dev/last_accuracy']
df = df[cols]

In [8]:
df = df.loc[df['dev/last_accuracy'] == 1.0]
df = df.loc[df['max_steps'] == 30]
df = df.loc[df['hidden_activation'] == "relu"]
df = df.loc[df['rnn_type'] == "jordan"]

In [9]:
print("Total df.shape:", df.shape)

Total df.shape: (314, 15)


In [10]:
df_add_cp9_h24 = df.query('operator == "add"').query('confidence_prob == 0.9').query('hidden_dimension == 24')
df_sub_cp9_h24 = df.query('operator == "subtract"').query('confidence_prob == 0.9').query('hidden_dimension == 24')

In [11]:
print(df_add_cp9_h24.shape)

(300, 15)


In [12]:
print(df_sub_cp9_h24.shape)

(14, 15)


In [13]:
n_rows = 300
df_add_cp9_h24 = get_partial_df(df_add_cp9_h24, n_rows)
print(df_add_cp9_h24.shape)

(300, 15)


# Export as CSV

## Functionalize

In [14]:
import pandas as pd
import numpy as np
import data_utils 
import config
from utils import create_dir
from os.path import join

def get_csv_df(df, filename, experiment_name):
    # Get configurations
    n_rows = df.shape[0]
    operator = df['operator'].iloc[0]
    operand_digits = df['operand_bits'].iloc[0]
    carry_list = list(data_utils.import_carry_datasets(operand_digits, operator).keys())
    
    # Gather for each 
    csv_df_list = list()
    for carries in carry_list:
        col = 'dev/last_carry-{}_mean_correct_answer_step'.format(carries)
        csv_df = pd.DataFrame(data={'mean_anwer_steps':df[col], 'carries':np.full((n_rows), carries)})
        csv_df_list.append(csv_df)
    csv_df = pd.concat(csv_df_list, ignore_index=True)
    
    # Change the order of columns
    csv_df = csv_df[['mean_anwer_steps', 'carries']]
    
    # Create dir
    dir_to_save = join(config.dir_result_statistics(), experiment_name)
    create_dir(dir_to_save)
    
    # Save the dataframe to a CSV file.
    csv_df.to_csv(join(dir_to_save, filename), index=False)

In [15]:
get_csv_df(df_add_cp9_h24, 'df_add_cp9_h24.csv', experiment_name)